# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score



In [2]:
df = pd.read_csv('previsao_de_renda.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
 # 1. Separando a base em treinamento
df_train, df_test = train_test_split(df, test_size=0.25, random_state=42)

# Testando
modelo = '''renda ~ sexo 
                    + posse_de_veiculo 
                    + posse_de_imovel 
                    + qtd_filhos 
                    + tipo_renda 
                    + educacao 
                    + estado_civil 
                    + tipo_residencia 
                    + idade 
                    + tempo_emprego 
                    + qt_pessoas_residencia
                    '''

y_train, X_train = patsy.dmatrices(modelo, data=df_train)
y_test, X_test = patsy.dmatrices(modelo, data=df_test)

In [5]:
# Rodando uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avaliando o $R^2$ na base de testes. 

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized('elastic_net', 
                             refit=True, 
                             L1_wt=0,  
                             alpha=alpha)
   
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha} - R2 = {r2}\n')


Alpha 0 - R2 = 0.26243403442818525

Alpha 0.001 - R2 = 0.26226194952350645

Alpha 0.005 - R2 = 0.2619663776008283

Alpha 0.01 - R2 = 0.261568475858286

Alpha 0.05 - R2 = 0.25638277607326476

Alpha 0.1 - R2 = 0.24863408637050977



A partir dessa regularização, o modelo 'alpha = 0.000' foi o que apresentou melhor coeficiente de determinação, sendo assim o melhor modelo.

In [6]:
# Fazendo o mesmo que no passo 2, com uma regressão *LASSO*.

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized('elastic_net', 
                             refit=True, 
                             L1_wt=1,  
                             alpha=alpha)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha} - R2 = {r2}\n')

Alpha 0 - R2 = 0.24863408637050977

Alpha 0.001 - R2 = 0.24863408637050977

Alpha 0.005 - R2 = 0.24863408637050977

Alpha 0.01 - R2 = 0.24863408637050977

Alpha 0.05 - R2 = 0.24863408637050977

Alpha 0.1 - R2 = 0.24863408637050977



In [7]:
# Rodando um modelo *stepwise*. Avaliando o $R^2$ na vase de testes.
df_dummies = pd.get_dummies(data=df.dropna(), drop_first=True).astype(int)

X = df_dummies.drop(columns='renda')
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    
    included = list(initial_list)
    while True:
        changed=False
       
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            print(included+[new_column])
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
       
        # Usando todos os coeficientes exceto o intercepto
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() 
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included




In [8]:
variaveis = stepwise_selection(X_test, y_test)
X = df_dummies[variaveis]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

y_pred = sm.OLS(y_train, X_train).fit().predict(X_test)

['data_ref_2015-12-01']
['posse_de_imovel']
['estado_civil_Solteiro']
['data_ref_2015-03-01']
['data_ref_2015-11-01']
['data_ref_2015-06-01']
['posse_de_veiculo']
['tipo_residencia_Governamental']
['tipo_residencia_Comunitário']
['educacao_Secundário']
['tipo_residencia_Estúdio']
['estado_civil_União']
['id_cliente']
['idade']
['data_ref_2015-07-01']
['data_ref_2015-08-01']
['data_ref_2015-10-01']
['tipo_renda_Servidor público']
['tempo_emprego']
['qtd_filhos']
['data_ref_2016-02-01']
['tipo_residencia_Casa']
['tipo_residencia_Com os pais']
['Unnamed: 0']
['tipo_renda_Pensionista']
['data_ref_2015-09-01']
['qt_pessoas_residencia']
['estado_civil_Viúvo']
['estado_civil_Separado']
['data_ref_2016-01-01']
['tipo_renda_Empresário']
['data_ref_2015-02-01']
['data_ref_2015-05-01']
['data_ref_2015-04-01']
['data_ref_2016-03-01']
['tipo_renda_Bolsista']
['educacao_Superior incompleto']
['educacao_Pós graduação']
['educacao_Superior completo']
['sexo_M']
Add  tempo_emprego                  with

In [12]:
# Regressão Ridge

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
y_r2 = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  
                             alpha=alpha)
    
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    y_r2.append(r2)
    print(f'Alpha {alpha} -  R2 = {r2}\n')
    


Alpha 0 -  R2 = 0.29011839253207306

Alpha 0.001 -  R2 = 0.29023801359372225

Alpha 0.005 -  R2 = 0.2906449281127462

Alpha 0.01 -  R2 = 0.29100733588363525

Alpha 0.05 -  R2 = 0.2900777188993673

Alpha 0.1 -  R2 = 0.2845177903932634



In [14]:
# Regressão Lasso

alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
y_r2 = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,  
                             alpha=alpha)
    
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    y_r2.append(r2)
    print(f'Alpha {alpha} - R-squared = {r2}\n')
    


Alpha 0 - R-squared = 0.29011839253207306

Alpha 0.001 - R-squared = 0.29011839253207306

Alpha 0.005 - R-squared = 0.29011839253207306

Alpha 0.01 - R-squared = 0.29011839253207306

Alpha 0.05 - R-squared = 0.29011839253207306

Alpha 0.1 - R-squared = 0.29011839253207306



Comparando as regresões dos modelos stepwise, apesar de discreta a diferença, a regressão Lasso apresentou o coeficiente de determinação maior e consequente melhor modelo para a análise. 

In [ ]:
# Comparando os parâmetros e avaliando eventuais diferenças.

Com dito antereiormente, o melhor modelo foi o de regressão Lasso, apesar da pouca variação.


In [18]:
# Tentando melhorar o  𝑅2 na base de testes.

df_matrix = df_dummies.loc[:, variaveis + ['renda']]

df_train, df_test = train_test_split(df_matrix, test_size=0.25, random_state=42)

modelo = smf.ols(formula='''np.log(renda) ~ tempo_emprego 
                                         + sexo_M 
                                         + idade 
                                         - posse_de_veiculo 
                                         - tipo_residencia_Estúdio
                                         ''', data=df_train)

reg = modelo.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,
                             alpha=0)

print(reg.summary())



                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.337
Method:                 Least Squares   F-statistic:                     1184.
Date:                Wed, 21 Feb 2024   Prob (F-statistic):               0.00
Time:                        16:19:05   Log-Likelihood:                -10312.
No. Observations:                9320   AIC:                         2.063e+04
Df Residuals:                    9316   BIC:                         2.067e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         7.3537      0.035    207.165

In [19]:
# Ajustando uma árvore de regressão e vendo se consigo um  𝑅2 melhor com ela.

X = df_dummies[variaveis]
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

reg_tree = DecisionTreeRegressor(random_state=42, max_depth=5, min_samples_leaf=13)
reg_tree.fit(X_train, y_train)

print('R2:', reg_tree.score(X=X_test, y=y_test))

R2: 0.38992777641514975
